In [1]:
import pandas as pd
import numpy as np
from catboost import Pool, CatBoostClassifier
import json
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score, roc_auc_score

In [2]:
data = json.load(open('RuBQ_2.0_test.json', encoding='utf-8'))

FileNotFoundError: [Errno 2] No such file or directory: 'RuBQ_2.0_test.json'

In [3]:
texts = [x['question_text'] for x in data]

In [4]:
df = pd.read_excel('concat_df.xlsx', index_col=0)

In [5]:
questions = df[0].to_list() + texts

In [6]:
answers = [1] * len(df) + [0] * len(texts)

In [7]:
list4df = [[x, y] for x, y in zip(questions, answers)]

In [8]:
new_df = pd.DataFrame(list4df, columns=['q', 'a'])

In [9]:
new_df

,q,a
0,Какие конкретные проекты по строительству и ре...,1
1,Каковы планы по расширению сети общественного ...,1
2,Как муниципалитет собирается справляться с про...,1
3,Как осуществляется мониторинг качества обществ...,1
4,Как решаются вопросы связанные с пересадками м...,1
...,...,...
2691,Какой высоты Ульмский собор в Германии?,0
2692,В какой стране зародился дадаизм?,0
2693,В каком году сняли с производства Jaguar E-type?,0
2694,С какого года закончили выпускать Rolls-Royce ...,0


In [10]:
text_features = ['q']

In [15]:
X_train, X_test, y_train, y_test = train_test_split(new_df['q'], new_df['a'])

In [19]:
learn_pool = Pool(
        X_train, 
        y_train, 
        text_features=text_features,
        feature_names=['q']
    )

In [33]:
model = CatBoostClassifier(iterations=3000, learning_rate=0.03, sampling_frequency='PerTree', task_type='GPU', class_weights=[0.2, 0.8])
model.fit(learn_pool, verbose=True)

0:	learn: 0.6315655	total: 20.9ms	remaining: 1m 2s
1:	learn: 0.5785990	total: 36.3ms	remaining: 54.5s
2:	learn: 0.5259620	total: 55.7ms	remaining: 55.6s
3:	learn: 0.4838749	total: 69.8ms	remaining: 52.3s
4:	learn: 0.4386730	total: 89.5ms	remaining: 53.6s
5:	learn: 0.4005990	total: 106ms	remaining: 52.9s
6:	learn: 0.3640589	total: 120ms	remaining: 51.5s
7:	learn: 0.3346320	total: 136ms	remaining: 51s
8:	learn: 0.3051936	total: 153ms	remaining: 50.8s
9:	learn: 0.2821823	total: 169ms	remaining: 50.4s
10:	learn: 0.2609136	total: 186ms	remaining: 50.4s
11:	learn: 0.2416427	total: 200ms	remaining: 49.8s
12:	learn: 0.2243335	total: 215ms	remaining: 49.3s
13:	learn: 0.2081375	total: 231ms	remaining: 49.2s
14:	learn: 0.1949914	total: 247ms	remaining: 49.2s
15:	learn: 0.1835025	total: 263ms	remaining: 49s
16:	learn: 0.1726310	total: 278ms	remaining: 48.7s
17:	learn: 0.1629576	total: 292ms	remaining: 48.4s
18:	learn: 0.1527480	total: 308ms	remaining: 48.3s
19:	learn: 0.1435437	total: 325ms	remain

In [37]:
y_pred = []
y_pred_proba = []
for el in X_test:
    y_pred.append(model.predict([el]))
    y_pred_proba.append(model.predict_proba([el]))

In [38]:
accuracy_score(y_test, y_pred), f1_score(y_test, y_pred), roc_auc_score(y_test, y_pred)

(0.9925816023738873, 0.9735449735449735, 0.9824834105990363)

In [39]:
model.save_model('text_clf.cbm')